# Modules

The Modules module permits to work directly with the GRASS GIS module.

More info about <a href="http://grass.osgeo.org/grass71/manuals/libpython/pygrass.modules.interface.html#module-pygrass.modules.interface.module" target="_blank">Module class</a>

In [ ]:
from IPython.display import Image
from pprint import pprint
from subprocess import PIPE

import numpy as np

from show import show


## Using the pygrass module interface

User who just wants to replace BASH with python has to import modules in the following manner:

In [ ]:
from grass.pygrass.modules.shortcuts import raster as r, vector as v, general as g, display as d
from grass.pygrass.modules import Module

And then call the grass modules

In [ ]:
g.region(raster='elevation', flags='p')

In [ ]:
print(g.region(flags='p', stdout_=PIPE).outputs.stdout)

If the name of the grass module contains "." you should replace it with "_". See the example bellow:

In [ ]:
r.slope_aspect(elevation='elevation', slope='slope', aspect='aspect', overwrite=True)
r.mapcalc("slope_gt_10 = if(slope > 10, slope, null())", overwrite=True)
print(r.info(map='slope_gt_10', flags='r', stdout_=PIPE).outputs.stdout)

In [ ]:
show('slope_gt_10', flags='n')

In [ ]:
show('slope')

Do something with the vectors:

In [ ]:
print(g.list(type='raster', flags='m', stdout_=PIPE).outputs.stdout)

In [ ]:
v.to_rast(input='hospitals', output='hospitals', type="point", use='cat', overwrite=True, quiet=True)

In [ ]:
print(g.list(type='raster', flags='m', stdout_=PIPE).outputs.stdout)

## Backwards compatibility

If you have a python script and want to convert it to use the new python interface, you just need to transform:

In [ ]:
#from grass.script.core import run_command
from grass.pygrass.modules import Module as run_command

run_command("r.info", map='elevation')
#etc...

## Module as object

Import the Module class, and instantiate the command, the name of the GRASS module is required.

In [ ]:
from grass.pygrass.modules import Module

slp = Module("r.slope.aspect")

### Attributes

In [ ]:
slp.name

In [ ]:
slp.description

In [ ]:
slp.keywords

In [ ]:
slp.label

In [ ]:
# get the documentation of the module
slp?

The instantiated module splits the parameters in:

* inputs
* outputs

both parameters are an [OrderedDict](http://docs.python.org/2/library/collections.html#collections.OrderedDict) python object. Additionally, the 'required' attribute returns a list of the parameters that are required.



In [ ]:
pprint(dict(slp.inputs))

In [ ]:
pprint(dict(slp.outputs))

In [ ]:
slp.required

In [ ]:
slp.inputs['elevation']

In [ ]:
slp.inputs.elevation = "elevation"

In [ ]:
slp.inputs.elevation

In [ ]:
slp.inputs["format"]

In [ ]:
slp.inputs.format = 'percents'

In [ ]:
slp.inputs.format = 'percent'

In [ ]:
slp.outputs.slope = 'slope'

In [ ]:
slp.flags.g = True

In [ ]:
slp.flags.a = True

In [ ]:
print slp.flags.overwrite # --overwrite, --verbose, --quite

In [ ]:
slp.flags.overwrite = True

# run the module
slp.run()

### Methods

In [ ]:
print slp  # __str__

In [ ]:
repr(slp)  # __repr__

In [ ]:
from grass.pygrass.modules import Module

slp = Module("r.slope.aspect")

slp(elevation='elevation', 
    slope='slp',  aspect='asp', 
    format='percent', overwrite=True, 
    verbose=False, flags='a', run_=False)

print slp.get_bash()
print slp.get_python()

## Run as a function

In [ ]:
slp(elevation='elevation', slope='slp',  aspect='asp', format='percent', overwrite=True)

## Special parameters

### "run_" to run later

In [ ]:
slp(elevation='elevation', slope='slp',  aspect='asp', format='percent', overwrite=True, run_=False)

### "finish_" to manage the process

In [ ]:
slp(elevation='elevation', slope='slp',  aspect='asp', format='percent', overwrite=True, run_=True, finish_=False)

In [ ]:
slp.popen.wait() # .kill()

### `stdin_`, `stdout_`, `stderr_`

In [ ]:
slp(elevation='elevation', slope='slp', aspect='asp', 
    format='percent', overwrite=True, stderr_=PIPE, run_=True)

In [ ]:
print(slp.outputs.stderr)

In [ ]:
gmapset = g.mapset(mapset='_', flags='p', stdout_=PIPE)
gmapset.outputs.stdout.strip()

# Summary

We have seen:

* how we can update existing Python script to use pygrass;
* play with the grass modules as objects, changing only few parameters;
* give more power to the user with the special parameters "*_".

For more info about Module you can see the <a href="http://grass.osgeo.org/grass71/manuals/libpython/pygrass.modules.interface.html#module-pygrass.modules.interface.module" target="_blank">class documentation</a>

# Exercise

## Time for coding!

In [ ]:
!g.region -p

Read the output of g.region and return a dictionary.

## Create a GRASS module using pygrass

In [ ]:
%%file mygrassmodule.py
#!/usr/bin/env python
# -- coding: utf-8 --
#
############################################################################
#
# MODULE:	    <name of your module>
#
# AUTHOR(S):    <your name>
#
# COPYRIGHT:	(C) 2013 by the GRASS Development Team
#
#		This program is free software under the GNU General Public
#		License (>=v2). Read the file COPYING that comes with GRASS
#		for details.
#
#############################################################################

##-----------------------
## DESCRIPTION
##-----------------------
#%Module
#%  description: <description>
#%  keywords: <keyword 1>
#%  keywords: <keyword 1>
#%  keywords: <keyword 1>
#%  overwrite: yes
#%End

##-----------------------
## VECTOR INPUT
##-----------------------
#%option G_OPT_V_MAP
#%  key: key0
#%  description: <parameter description>
#%  required: yes
#%end

##-----------------------
## MULTI RASTER INPUT
##-----------------------
#%option G_OPT_R_INPUTS
#%  key: key1
#%  description: <parameter description>
#%  multiple: yes
#%  required: no
#%end

##-----------------------
## STRING INPUT
##-----------------------
#%option
#%  key: key2
#%  description: <parameter description>
#%  type: string
#%  multiple: yes
#%  required: no
#%end
from pprint import pprint
from grass.script import parser

def main(opts, flgs):
    print('OPTIONS:')
    pprint(opts)
    print('FLAGS:')
    pprint(flgs)
    # do something with GRASS script

if __name__ == "__main__":
    opts, flgs = parser()
    main(opts, flgs)



In [ ]:
!python2 mygrassmodule.py --h

In [ ]:
!python2 mygrassmodule.py key0=pippo key1=pluto,topolino key2=minni,clarabella